# EC Cryptography

![ec](./elliptic-curve-add.png)

Elliptical curve cryptography (ECC) is an area of public key cryptography (not a specific algorithm) based on elliptic curve theory that can be used to create faster, smaller, and more efficient cryptographic keys. ECC generates keys through the properties of the elliptic curve equation instead of the traditional method of generation as the product of very large prime numbers.  Ref: https://uwillnvrknow.github.io/deCryptMe/pages/ellipticCurve.html

The way EC keys are used is similar to RSA with the exception that persistent re-usable keys do not provide for good forward security. So we tend to not encrypt directly with our EC keys. We do use ephemeral EC keys when exchanging a Symmetric key **ECDHE** and when Signing with **ECDSA.**

## ECHDE
EC Diffie Hellman Exhange is a key exchange mechanism that uses ephemeral EC keys to exchange a robust secret for mutual use.

Traditional DH exchange uses the principles used by RSA to share public key components (which are prime exponents) whereas ECDHE exchanges ephemeral (generated just for this purpose) EC public keys (which are just points on an elliptic curve) using the following logic.  These keys are used by each party to create a common key (integer pair) by multiply each others public key with their own private key (the x value of another point on the elliptic curve). EC mathematical magic is such that Alice's (a,b) shares her public key b with Bob (c,d) who in turn creates m=b.c and Bob in turns shares his public key d and Alice creates m=d.a

Now they both have a key not known to anybody else called m, which they may use to exchange further information securely.


> `sudo python3 -m pip install tinyec`

In [2]:
from tinyec import registry
import secrets

# registry.EC_CURVE_REGISTRY

In [4]:
def compress(pubKey):
    return hex(pubKey.x) + hex(pubKey.y % 2)[2:]

curve = registry.get_curve('brainpoolP256r1')

alicePrivKey = secrets.randbelow(curve.field.n)
alicePubKey = alicePrivKey * curve.g
print("Alice public key:", compress(alicePubKey))

bobPrivKey = secrets.randbelow(curve.field.n)
bobPubKey = bobPrivKey * curve.g
print("Bob public key:", compress(bobPubKey))

print("Now exchange the public keys (e.g. through Internet)")

aliceSharedKey = alicePrivKey * bobPubKey
print("Alice shared key:", compress(aliceSharedKey))

bobSharedKey = bobPrivKey * alicePubKey
print("Bob shared key:", compress(bobSharedKey))

print("Equal shared keys:", aliceSharedKey == bobSharedKey)

Alice public key: 0x6c108ebac54f5b7a1829c4bbe31791debfa63e5e69ce026e61b7265885915c70
Bob public key: 0x91dc69d4075a8ba81511eeb18507c505dc9a2534dcbb5ed68ee637863121e3591
Now exchange the public keys (e.g. through Internet)
Alice shared key: 0x247ca108bb540358a9f1e4c049dfc871e1d9011be2471a2cd88e66add14bf45d1
Bob shared key: 0x247ca108bb540358a9f1e4c049dfc871e1d9011be2471a2cd88e66add14bf45d1
Equal shared keys: True


In practice this calculated shared key would then be passed through a key spreading algorithm such as `hkdef` of `pbkdf2` for additional strength and to ensure the key length was always as required.

# ECDSA

Elliptic Curve Digital Signature Algorithm uses the properties of Elliptic Curves to allow for signatures and signature verification, so without drowing in the mathematics, we shall use the Python package, which implements the ECDSA signature algorithm with the curve secp256k1 (used in the Bitcoin cryptography), as well as many other functionalities related to the Bitcoin blockchain.

> `python3 -m pip install ecpy`

In [6]:
from ecpy.curves     import Curve,Point
from ecpy.keys       import ECPublicKey, ECPrivateKey
from ecpy.ecdsa      import ECDSA

cv     = Curve.get_curve('secp256k1')
pu_key = ECPublicKey(Point(0x65d5b8bf9ab1801c9f168d4815994ad35f1dcb6ae6c7a1a303966b677b813b00,

                           0xe6b865e529b8ecbf71cf966e900477d49ced5846d7662dd2dd11ccd55c0aff7f,
                           cv))
pv_key = ECPrivateKey(0xfb26a4e75eec75544c0f44e937dcf5ee6355c7176600b9688c667e5c283b43c5,
                      cv)


signer = ECDSA()
# we typically sign a hash of the message as the message may be too long..
sig    = signer.sign(b'01234567890123456789012345678912',pv_key)

# we may safely distribute our EC public key as we don't use the corresponding provate key for anything else except signing
assert(signer.verify(b'01234567890123456789012345678912',sig,pu_key))

## openssl and ECDSA

It is extremely useful as a developer to be able to cross reference your work in code with the openssl library by using the cli.

So lets create some EC keys and do some signing at the CLI

```
openssl ecparam -name prime256v1 -genkey -out sk.pem
openssl ec -in sk.pem -pubout -out vk.pem
echo "data for signing" > data
openssl dgst -sha256 -sign sk.pem -out data.sig data
openssl dgst -sha256 -verify vk.pem -signature data.sig data
openssl dgst -sha256 -prverify sk.pem -signature data.sig data
```

now lets do the same in code using the `ecdsa` library.

> sudo python3 -m pip install ecdsa


In [8]:
import hashlib
from ecdsa import SigningKey, VerifyingKey
from ecdsa.util import sigencode_der, sigdecode_der

with open("vk.pem") as f:
   vk = VerifyingKey.from_pem(f.read())

with open("data", "rb") as f:
   data = f.read()

with open("data.sig", "rb") as f:
   signature = f.read()

assert vk.verify(signature, data, hashlib.sha256, sigdecode=sigdecode_der)

with open("sk.pem") as f:
   sk = SigningKey.from_pem(f.read(), hashlib.sha256)

new_signature = sk.sign_deterministic(data, sigencode=sigencode_der)

with open("data.sig2", "wb") as f:
   f.write(new_signature)


and check the final sig2 with `openssl dgst -sha256 -verify vk.pem -signature data.sig2 data`
and alwys interesting to see how an ec key is structured in a PEM file `openssl ec -in sk.pem -text -noout`